### Using pytorch kinematics
1. This is based on the repository from https://github.com/UM-ARM-Lab/pytorch_kinematics
2. Simply install and run `pip-compile requirements.toml` and `pip install -r requirements.txt`

In [2]:
import math
import torch
import pytorch_kinematics as pk

import os
from timeit import default_timer as timer

import time

d = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

chain = pk.build_serial_chain_from_urdf(open("./urdf/kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
chain = chain.to(dtype=dtype, device=d)

N = 1000
th_batch = torch.rand(N, len(chain.get_joint_parameter_names()), dtype=dtype, device=d)
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)

########################### FK ###########################

t1 = timer()
# order of magnitudes faster when doing FK in parallel
# elapsed 0.008678913116455078s for N=1000 when parallel
# (N,4,4) transform matrix; only the one for the end effector is returned since end_only=True by default
tg_batch = chain.forward_kinematics(th_batch)

t2 = timer()

# elapsed 8.44686508178711s for N=1000 when serial
for i in range(N):
    tg = chain.forward_kinematics(th_batch[i])
t3 = timer()

print("Execution time for FK, [parallel] = {}, [sequential] = {}".format(t2-t1, t3-t2))

# ########################### JACOBIAN ###########################

# t1 = timer()
# J = chain.jacobian(th)
# t2 = timer()

# print("Execution time for Jac, [parallel] = {}".format(t2-t1))

# ########################### INVERSE KINEMATICS ###########################
# # transformation from robot base frame to world frame
# pos = torch.tensor([0.0, 0.0, 0.0], device=d)
# rot = torch.tensor([0.0, 0.0, 0.0], device=d)
# rob_tf = pk.Transform3d(pos=pos, rot=rot, dtype=dtype, device=d)

# # world frame goals
# # generate random goal joint angles (so these are all achievable)
# # use the joint limits to generate random joint angles
# lim = torch.tensor(chain.get_joint_limits(), device=d)
# goal_q = torch.rand(N, 7, dtype=dtype, device=d) * (lim[1] - lim[0]) + lim[0]

# # get ee pose (in robot frame)
# goal_in_rob_frame_tf = chain.forward_kinematics(goal_q)


# frame_indices = chain.get_all_frame_indices()
# print("frame indices = {}".format(frame_indices))

# # transform to world frame for visualization
# goal_tf = rob_tf.compose(goal_in_rob_frame_tf)
# goal = goal_tf.get_matrix()
# goal_pos = goal[..., :3, 3]
# goal_rot = pk.matrix_to_euler_angles(goal[..., :3, :3], "XYZ")

# ik = pk.PseudoInverseIK(chain, max_iterations=30, num_retries=10,
#                         joint_limits=lim.T,
#                         early_stopping_any_converged=True,
#                         early_stopping_no_improvement="all",
#                         # line_search=pk.BacktrackingLineSearch(max_lr=0.2),
#                         debug=False,
#                         lr=0.2)

# t1 = timer()
# sol = ik.solve(goal_in_rob_frame_tf)
# t2 = timer()
# print("Execution time for IK, [parallel] = {}".format(t2-t1))
# print("IK converged number: %d / %d" % (sol.converged.sum(), sol.converged.numel()))
# print("IK took %d iterations" % sol.iterations)
# print("IK solved %d / %d goals" % (sol.converged_any.sum(), N))

Execution time for FK, [parallel] = 0.0018339760063099675, [sequential] = 1.022412815997086


In [3]:
import math
import torch
import pytorch_kinematics as pk

import os
from timeit import default_timer as timer

import time

d = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

chain = pk.build_serial_chain_from_urdf(open("./urdf/kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
# chain = chain.to(dtype=dtype, device=d)

th = torch.tensor([0.0, -math.pi / 4.0, 0.0, math.pi / 2.0, 0.0, math.pi / 4.0, 0.0])
th = th.repeat(2, 1)
loc = torch.tensor([[0.1, 0, 0], [-0.1, 0.05, 0]])
# loc = torch.tensor([0.1, 0, 0])
J = chain.jacobian(th, locations=loc)

th
loc

joint name = lbr_iiwa_joint_1, joint rpy = [0.0, 0.0, 0.0], joint xyz = [0.0, 0.0, 0.1575]
link name = lbr_iiwa_link_1
joint name = lbr_iiwa_joint_2, joint rpy = [1.57079632679, 0.0, 3.14159265359], joint xyz = [0.0, 0.0, 0.2025]
link name = lbr_iiwa_link_2
joint name = lbr_iiwa_joint_3, joint rpy = [1.57079632679, 0.0, 3.14159265359], joint xyz = [0.0, 0.2045, 0.0]
link name = lbr_iiwa_link_3
joint name = lbr_iiwa_joint_4, joint rpy = [1.57079632679, 0.0, 0.0], joint xyz = [0.0, 0.0, 0.2155]
link name = lbr_iiwa_link_4
joint name = lbr_iiwa_joint_5, joint rpy = [-1.57079632679, 3.14159265359, 0.0], joint xyz = [0.0, 0.1845, 0.0]
link name = lbr_iiwa_link_5
joint name = lbr_iiwa_joint_6, joint rpy = [1.57079632679, 0.0, 0.0], joint xyz = [0.0, 0.0, 0.2155]
link name = lbr_iiwa_link_6
joint name = lbr_iiwa_joint_7, joint rpy = [-1.57079632679, 3.14159265359, 0.0], joint xyz = [0.0, 0.081, 0.0]
link name = lbr_iiwa_link_7
idx = 0, parents_indices = [[0]]
idx = 1, parents_indices = [[0], 

tensor([[ 0.1000,  0.0000,  0.0000],
        [-0.1000,  0.0500,  0.0000]])

In [1]:
import math
import torch
import pytorch_kinematics as pk

data = '<robot name="test_robot">' \
           '<link name="link1" />' \
           '<link name="link2" />' \
           '<link name="link3" />' \
           '<joint name="joint1" type="revolute">' \
           '<origin xyz="1.0 0.0 0.0"/>' \
           '<parent link="link1"/>' \
           '<child link="link2"/>' \
           '</joint>' \
           '<joint name="joint2" type="revolute">' \
           '<origin xyz="1.0 0.0 0.0"/>' \
           '<parent link="link2"/>' \
           '<child link="link3"/>' \
           '</joint>' \
           '</robot>'

chain = pk.build_serial_chain_from_urdf(data, 'link3')
# th1 = torch.tensor([0.42553542, 0.17529176])
# tg = chain.forward_kinematics(th1)

th_batch = torch.rand(4, 2)
tg_batch = chain.forward_kinematics(th_batch)

joint name = joint1, joint rpy = [0, 0, 0], joint xyz = [1.0, 0.0, 0.0]
link name = link2
joint name = joint2, joint rpy = [0, 0, 0], joint xyz = [1.0, 0.0, 0.0]
link name = link3
Frame indices = tensor([2]), th = tensor([[0.1228, 0.1196],
        [0.9231, 0.7872],
        [0.5903, 0.0962],
        [0.9480, 0.5405]])
Axis [shape=torch.Size([4, 2, 3])]:
 tensor([[[0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [0., 0., 1.]]])
th=tensor([[0.1228, 0.1196],
        [0.9231, 0.7872],
        [0.5903, 0.0962],
        [0.9480, 0.5405]]), torch.Size([4, 2])
